In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
from tqdm import tqdm,tnrange,tqdm_notebook
import librosa
import librosa.display
from PIL import Image
import shutil
import warnings
warnings.filterwarnings(action='ignore')
from mpl_toolkits.axes_grid1 import ImageGrid

''' HELPER FUNCTIONS '''

# Just Seaborn Barplot w/ Common Input Format
def bar_plot(x, y,title='Training Soundscape : 5 second segment identification', xlim = None, ylim = None, 
             xticklabels = None, yticklabels = None,xlabel = None, ylabel = None, 
             figsize = (10,4),axis_grid = 'x',xrotation=None, yrotation=None ):
        
    cmap = sns.color_palette("mako")
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title)

    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(True);ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False);ax.spines['left'].set_visible(False)

    sns.barplot(x = x, y = y, edgecolor = 'black', ax = ax,palette = cmap)
    ax.set_xlim(xlim);ax.set_ylim(ylim)    
#     ax.set_xticklabels(xticklabels);ax.set_yticklabels(yticklabels)
    plt.xlabel(xlabel);plt.ylabel(ylabel)
    ax.grid(axis = axis_grid,ls='--',alpha = 0.3)
    plt.xticks(rotation=xrotation)
    plt.yticks(rotation=yrotation)
    plt.savefig('bar_out.png');plt.show();
    
def show_grid(image_list,nrows,ncols,label_list=None,show_labels=False,savename=None,figsize=(20,10),showaxis='off'):
    if type(image_list) is not list:
        if(image_list.shape[-1]==1):
            image_list = [image_list[i,:,:,0] for i in range(image_list.shape[0])]
        elif(image_list.shape[-1]==3):
            image_list = [image_list[i,:,:,:] for i in range(image_list.shape[0])]
    fig = plt.figure(None, figsize,frameon=False)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(nrows, ncols),  # creates 2x2 grid of axes
                     axes_pad=0.3,  # pad between axes in inch.
                     share_all=True,
                     )
    for i in range(nrows*ncols):
        ax = grid[i]
        img = Image.open(image_list[i])
        ax.imshow(img,cmap='Greys_r')  # The AxesGrid object work as a list of axes.
        ax.axis('off')
        if show_labels:
            ax.set_title(class_mapping[y_int[i]])
    if savename != None:
        plt.savefig(savename,bbox_inches='tight')
        
''' Extracts spectrograms and saves them in a working directory '''
def get_spectrograms(filepath, primary_label, output_dir):

    # load one signal via librosa
    # split signal into parts, storing them in a list

    sig, rate = librosa.load(filepath, sr=cfg.sr, offset=None, duration=cfg.cutoff)
    sig_splits = split_signal(sig) # split the signal into parts (exactly 5s segments)

    # Extract mel spectrograms for each audio chunk
    s_cnt = 0
    saved_samples = []
    for chunk in sig_splits:

        s_cnt += 5
        
        hop_length = int(cfg.sl * cfg.sr / (cfg.sshape[1] - 1))
        mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                                  sr=cfg.sr, 
                                                  n_fft=1024, 
                                                  hop_length=hop_length, 
                                                  n_mels=cfg.sshape[0], 
                                                  fmin=cfg.fmin, 
                                                  fmax=cfg.fmax)
    
        mel_spec = librosa.power_to_db(mel_spec**2, ref=np.max) 
        
        # Normalize
        mel_spec -= mel_spec.min()
        mel_spec /= mel_spec.max()
        
        # Save as image file
        save_dir = os.path.join(output_dir, primary_label)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        save_path = os.path.join(save_dir, filepath.rsplit(os.sep, 1)[-1].rsplit('.', 1)[0] + 
                                 '_' + str(s_cnt) + '.png')
        im = Image.fromarray(mel_spec * 255.0).convert("L")
        im.save(save_path)        
        saved_samples.append(save_path)

    return saved_samples

''' CREATE SPECTOGRAMS FROM DATAFRAME '''
# Create Spetograms from dataframe, containing pathwys to short audio, .ogg
def df_to_spectogram(ldf):

    path_temp = DIR_SPEC_OUT + 'bird'
    if(os.path.exists(path_temp)):
        shutil.rmtree(path_temp)

    samples = []
    with tqdm_notebook(total=len(ldf)) as pbar:
        for idx, row in ldf.iterrows():   # cycle through each row in df
            pbar.update(1)

            audio_file_path = ldf.loc[idx,'path']
            samples += get_spectrograms(audio_file_path, row.primary_label,path_temp)
                
    print(f'CREATED # OF SPECTOGRAMS: {len(samples)} FILES')
    
''' Split librosa signal the into segments '''
def split_signal(sig):
    sig_splits = []
    for i in range(0, len(sig), int(cfg.sl * cfg.sr)):
        split = sig[i:i + int(cfg.sl * cfg.sr)]
        if len(split) < int(cfg.sl * cfg.sr):
            break
        sig_splits.append(split)
    
    return sig_splits

# <sub>I.</sub> <span style='color:#F7765E'><sub>SOUNDSCAPE DATA</sub></span>
#### <b>GENERAL OVERVIEW</b>
- Envronment recordings can be referred to as <b>soundscapes</b>; typically a long recording without labels.
<b>In this competition</b>:
- The one big recording is to be <b>split into 5 second segments</b> & for each segment we are required to indicate either of the two:
    - (a) whether a bird is present, and if so which one.
    - (b) no bird is present.

<b>Sample Recording</b>

Whether you analyse the recording in time domain only or split it into time & frequency domain, let's view what the recording would look like:

#### <b>train_short_audio RELATION</b>
- In this competition, we are given a set of short audio slips (located in folder: <b>train_short_audio</b>), all of which are all labeled.
- These collections of recordings contain audio signals of a particular specie with varying degree of background/foreground noise (<code>train_metadata.csv</code>'s <b>rating data</b>). The noise a bird makes should fundamentally be cleaner than in the soundscape recording. 
- In this competition, we are provided with a file <code>train_metadata.csv</code>, which contains the information about how clean each of these recordings is; <code>rating</code>.
- They are useful to extract patterns that can be present in the <b>soundscape data</b> we will be given.

#### <b>TRAINING & TEST RECORDING LOCATIONS</b>
We can extract location data from <code>txt</code> located in the <b>test_soundscapes</b> folder.

In [ ]:
tdf = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')

# latitude, longitude
COL = [5.57,-75.85,200,'Jardín, Departamento de Antioquia','Colombia']
COR = [10.12,-84.51,200,'Alajuela, San Ramón','Costa Rica']
SNE = [38.49,-119.95,200,'Sierra Nevada, California','USA'] 
SSW = [42.47,-76.45,200,'Ithaca, New York','USA']
index = ['COL','COR','SNE','SSW']
columns = ['latitude','longitude','size','location','country']
data = [COL,COR,SNE,SSW]
test_tdf = pd.DataFrame(data,index=index,columns=columns)
test_tdf

#### <b>TRAINING SOUNDSCAPE : COUNTS INFORMATION</b>
- Let's view the value counts difference between <b>nocall</b> and <b>call</b> labels in the <b>training soundscape</b> data.
- Splitting the whole soundscape recording into <b>5 second</b> segments, let's also see how many segments exist in each recording, in the <b>training soundscape</b> data.

In [ ]:
# Training Given Environment Recordings
tdf = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')
tdf0 = pd.read_csv('../input/birdclef-2021/train_metadata.csv')
val = tdf.birds.value_counts(); y = val.to_list(); x = val.index.to_list()

print('CALLS vs NOCALLS INFO in All Recordings')
print(f"Training Soundscape Identifiers: {tdf[tdf.birds!='nocall'].shape[0]}")
print(f"Training Soundscapes Nocalls: {tdf[tdf.birds=='nocall'].shape[0]}")

print('\nTRAINING SOUNDSCAPE RECORDINGS:')
tdf.site.value_counts() # 2/4 TEST LOCATIONS

#### <b>TRAINING SOUNDSCAPE BIRD COUNTS</b>
- We are given the labels for all <b>training soundscape</b> intervals in the file, <code>train_soundscape_labels.csv</code>
- Let's see which birds are present in these recordings at the two locations (<b>COR</b> & <b>SSW</b>)

In [ ]:
# Visualise the Birds present in Recordings
bar_plot(x=x[1:],y=y[1:],figsize=(20,5),xrotation=90,title='Soundscape Data: Birds Labeled in All Recordings')

#### <b>SOUNDSCAPE RECORDNG LOCATIONS</b>
- Haven been given the locations of each recording location, in the <b>test_soundscapes</b> folder, we know the locations of both <b>training</b> & <b>test</b> recording locations.
- Let's plot, both the soundscape locations (<b>orange</b>) & train_short_audio recording locations (<b>hexbin map</b>) which are from xeno-canto.

In [ ]:
# Unique Token ID
mapbox_access_token = 'pk.eyJ1Ijoic2h0cmF1c3NhcnQiLCJhIjoiY2tqcDU2dW56MDVkNjJ6angydDF3NXVvbyJ9.nx2c5XzUH9MwIv4KcWVGLA'
plot_hex = True
import plotly.figure_factory as ff
if(plot_hex):

    fig = ff.create_hexbin_mapbox(
        data_frame=tdf0, lat="latitude", lon="longitude",
        nx_hexagon=200, opacity=0.6,min_count=1, labels={"color": "Point Count"},
        color_continuous_scale="mint",
        range_color = [0,100],
        show_original_data=True, # show point data
        original_data_marker=dict(size=2, opacity=0.05,color='black'), # point data options
        zoom = 10,height=500)
    fig2 = go.Scattermapbox(
            lat=test_tdf['latitude'],
            lon=test_tdf['longitude'],
            text=[test_tdf['location'][i]+': Soundscape' for i in range(test_tdf.shape[0])],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=30,color='rgb(255, 87, 51)',opacity=0.3)
    )
    fig.add_trace(fig2)
    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,pitch=0,zoom=1))
    fig.update_layout(title="<b>Train Short Audio</b> | Soundscape Locations",
                      margin={"r":0,"t":80,"l":0,"b":0},mapbox_style="light");
    fig.update_layout(coloraxis_showscale=False,showlegend=False);fig.show()

#### <b>LOCATION SEGMENT LABELS: SSW </b>
- One of the locations available to us in the <b>train_soundscapes</b> data.
- Let's plot a heatmap of <b>all the soundscape segments (120 per recording)</b> & the <b>corresponding label</b> at location <b>SSW</b>.
- SSW: Ithaca, New York, USA Recording Sightings.

In [ ]:
lst_tdfs = []; ii=-1
for i in tdf.audio_id.value_counts().index:
    height_id = [2,1,1,2,2,
                 3,5,4,1.5,1.5]

    loc_tdf = tdf[tdf.audio_id == i].reset_index()
    if(loc_tdf.loc[0,'site']=='SSW'):
        loc_id = str(loc_tdf.loc[0,'site']) + '_' + str(+ loc_tdf.loc[0,'audio_id'])
        ii+=1; lst_tdfs.append(loc_tdf)

        # get heatmap
        heatmap = lst_tdfs[ii].pivot_table(index='birds',columns=lst_tdfs[ii].index+1,values='seconds')
        fig,ax = plt.subplots(figsize=(30,height_id[ii]))
        cmap = sns.color_palette("mako")
        sns.heatmap(heatmap,ax=ax,cbar=False,linecolor='k',lw=1,cmap=cmap)
        plt.yticks(rotation=0);plt.title(loc_id)
        plt.savefig(loc_id+'.png');
        plt.show()

#### <b>LOCATION SEGMENT LABELS: COR </b>
- The second & final location available to us in the <b>train_soundscapes</b> data.
- Let's plot a heatmap of <b>all the soundscape segments (120 per recording)</b> & the <b>corresponding label</b> at location <b>COR</b> as well.
- COR: Alajuela, San Ramón, Costa Rica Recording Sightings.

In [ ]:
lst_tdfs = []; ii=-1
for i in tdf.audio_id.value_counts().index:
    height_id = [0.5,3,3,0.5,2,
                 2,3,1,0.5,2]

    loc_tdf = tdf[tdf.audio_id == i].reset_index()
    if(loc_tdf.loc[0,'site']=='COR'):
        loc_id = str(loc_tdf.loc[0,'site']) + '_' + str(+ loc_tdf.loc[0,'audio_id'])
        ii+=1; lst_tdfs.append(loc_tdf)

        # get heatmap
        heatmap = lst_tdfs[ii].pivot_table(index='birds',columns=lst_tdfs[ii].index+1,values='seconds')
        fig,ax = plt.subplots(figsize=(30,height_id[ii]))
        cmap = sns.color_palette("mako")
        sns.heatmap(heatmap,ax=ax,cbar=False,linecolor='k',lw=1,cmap=cmap)
        plt.yticks(rotation=0);plt.title(loc_id)
        plt.savefig(loc_id+'.png');
        plt.show()

# <sub>II.</sub> <span style='color:#F7765E'><sub>SHORT RECORDING RELATIVE TO SOUNDSCAPE LOCATIONS</sub></span>

- Short Audio Clips in relation to <b>Training & Test Soundscape</b> locations. 
- Soundscape recording locations are not precise, unlike Short Audio clips, as indicated in this [reply](https://www.kaggle.com/c/birdclef-2021/discussion/232238):
> Each location description is just a rough estimation of the area that we used as recording site. We use recorder arrays which are often distributed across a habitat. So there is no precise location, I used the center of the recording cluster and then rounded lat and lon which still provides a good indication for a potential species mix.
- Let's plot the recorded locations of each bird specie (<b>primary</b>) as well as the <b>soundscape locations</b>, to do that we can create a small class to help us keep things clean.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns;sns.set(style='whitegrid')
from matplotlib import cm
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings(action='ignore')
import datetime
pd.set_option('display.max_columns', None) 
mapbox_access_token = 'pk.eyJ1Ijoic2h0cmF1c3NhcnQiLCJhIjoiY2tqcDU2dW56MDVkNjJ6angydDF3NXVvbyJ9.nx2c5XzUH9MwIv4KcWVGLA'
import plotly.figure_factory as ff
import plotly.graph_objects as go

# global config file
class config:
    base_dir = '/kaggle/input/birdclef-2021/'
    os.chdir(base_dir)
    print(f'cwd: {os.getcwd()}')

# class for dealing with soundscapes
class birdclef:

    def __init__(self):
        
        ''' SHORT TRAINING FILES '''
        self.path_short_audio = './train_short_audio/' 
        self.pd_short_audio = pd.read_csv('train_metadata.csv')
        self.pd_short_audio['path'] = self.path_short_audio + "/" + self.pd_short_audio['primary_label'] + '/' + self.pd_short_audio['filename']

        ''' TRAINING SOUNDSCAPE FILES '''
        self.so_path_tr = './train_soundscapes/'  # path to train soundcape files
        self.so_path_te = './test_soundscapes/'  # path to test soundcape files
        self.sscape_audio = pd.read_csv(config.base_dir+'train_soundscape_labels.csv')   # read soundscape related CSV

        lst_sounds = os.listdir(self.so_path_tr)
        self.path_scape = [self.so_path_tr + i for i in lst_sounds]
        self.path_scape.sort()
        
        ''' TEST SANDSCAPE INFORMATION '''
        COL = [5.57,-75.85,200,'Jardín, Departamento de Antioquia','Colombia']
        COR = [10.12,-84.51,200,'Alajuela, San Ramón','Costa Rica']
        SNE = [38.49,-119.95,200,'Sierra Nevada, California','USA'] 
        SSW = [42.47,-76.45,200,'Ithaca, New York','USA']
        index = ['COL','COR','SNE','SSW']
        columns = ['latitude','longitude','size','location','country']
        data = [COL,COR,SNE,SSW]
        self.df_tsi = pd.DataFrame(data,index=index,columns=columns)

    # Show DataFrame with relevant Training Soundcape Information
    def info_scape(self,path):
#         self.load_id = path.split('_')[1].split("\\")[1]  # get file identifier name from pathway
        self.load_id = path.split('_')[1].split("/")[1]  # get file identifier name from pathway

        tdf = self.sscape_audio[self.sscape_audio.audio_id == int(self.load_id)].copy()
        tdf['time'] = tdf.seconds.apply(lambda x: str(datetime.timedelta(seconds=x)))
        display(tdf.T)
        
    ''' [MAIN] TRAINING FILES RELATED STUFF'''

    # get various subsets of dataframe
    def get_bird_subset(self,name='acafly'):
        return self.pd_short_audio[self.pd_short_audio['primary_label'] == name].copy().reset_index()
    # get rating subset
    def get_rating_subset(self,rating=2):
        return self.pd_short_audio[self.pd_short_audio['rating'] == rating].copy().reset_index()
    # get bird & rating subset
    def get_bird_rating(self,name='acafly',rating=4):
        return self.pd_short_audio[(self.pd_short_audio['primary_label'] == name)&(self.pd_short_audio['rating'] == rating)].copy().reset_index()

    # get all birds available in short sound files
    def get_short_labels(self):
        primary_labels = self.pd_short_audio.primary_label.unique()
        primary_labels.sort()
        return primary_labels
    
    # Check where each primary label was observed
    def check_map(self,ldf,anim,bins):
        fig = ff.create_hexbin_mapbox(
            data_frame=ldf, lat="latitude", lon="longitude",nx_hexagon=bins,
            opacity=0.6,min_count=1, labels={"color": "Point Count"},
            color_continuous_scale="viridis",
            range_color = [0,10],
            show_original_data=True, # show point data
            original_data_marker=dict(size=2, opacity=0.05,color='black'),
            animation_frame = anim, # point data options
            zoom = 10,height=500)
        fig2 = go.Scattermapbox(
                lat=self.df_tsi['latitude'],
                lon=self.df_tsi['longitude'],
                text=[self.df_tsi['location'][i]+': Soundscape' for i in range(self.df_tsi.shape[0])],
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=30,color='rgb(255, 87, 51)',opacity=0.3)
        )
        fig.add_trace(fig2)
        fig.update_layout(
            hovermode='closest',
            mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,pitch=0,zoom=1))
        fig.update_layout(title="<b>Short Audio Bird Recording Locations</b> | Soundscape Locations",
                          margin={"r":0,"t":80,"l":0,"b":0},mapbox_style="light");
        fig.update_layout(coloraxis_showscale=False,showlegend=False);
        fig.layout.updatemenus[0].pad.t=40;fig.show()
        
main = birdclef()
main.pd_short_audio.head(1)

#### <b>RECORD PERIOD VARIATION</b>
- We can check for a given <b>subset</b>, for example, when were audio clips recorded, using the handy plotly map. 
- You can easily create your own functions by modifying the <b>class birdclef</b> above, if you need more specific combinations; 

Here are some to get us started (available in the class):
- <code>def get_bird_subset(self,name='acafly'):</code> to select a specific <b>primary</b> label (specie) subset.
- <code>def get_rating_subset(self,rating=2):</code> to select a specific rating subset.
- <code>def get_bird_rating(self,name='acafly',rating=4):</code> to select both a specific <b>primary label</b> & <b>rating</b>.

To use the class plot, let's simply call main.check_map(), having already instantiated the class, the function needs three arguments
- <b>dataframe</b> : any of the three above will do, eg. <b>main.get_bird_subset('littin1')</b>
- <b>animation column</b> : Desired animation column in the dataframe, eg. <b>'date'</b>
- <b>hexbin size</b> : Some adjustments of the hexbin size may be needed for better visualisation, eg. <b>80</b>

Let's try an example visualisation, in both cases I've limited the amount of data input into the current map to reduce the load on my end.

In [ ]:
main.get_short_labels()[:10] # get list of primary labels
main.check_map(main.get_bird_subset('littin1')[:100],'date',10)

#### <b>PRIMARY BIRD SPECIE VARIATION </b>
- Let's try another example, perhaps a more useful one.
- We can use the same animation feature, to cycle through all bird species and check their recorded locations.
- Let's check all recordings with a rating of <b>rating=4</b>, to do that we simply pass <b>'primary_label'</b> to the second argument & some fitting value for the <b>hexbin</b> count.

In [ ]:
main.check_map(main.get_rating_subset(rating=4)[:100],'primary_label',30)

# <sub>III.</sub> <span style='color:#F7765E'><sub>SOUNDSCAPE NOISE EXTRATOR</sub></span>
- The training soundscape recordings have been annotate by experts for each of the <b>120 segments x 5 seconds/ 20 recordings.</b>
- Some segments are <b>labelled as a specific primary_label</b> (including multiple primary labels) & others labelled as <b>nocalls</b>.
- Aside from creating noise during augmentation, recording overlapping & alike , we can utilise the soundscape data and extract <b>nocall</b> segments as well.

In [ ]:
class cfg:

    # Generate Subset
    rat_id = 4 # rating subset limiter 
    recs = 200 # each specie must have X recodings
    max_files = 1500 # general last limit for rows
    thresh = 0.25 # label probability selection threshold
    submission = True # For Submission Only (Less Inference Output)
    
    # Global vars
    seed = 1337
    sr = 32000        # librosa sample rate input
    sl = 5 # seconds   
    sshape = (48*3,128*3) # height x width
    fmin = 500      # spectrum min frequency
    fmax = 12500    # spectrum max frequency
    n_epoch = 100   # training epochs
    cutoff = 600     # 3 sample spectogram (training) overwritten for inference
    
os.chdir('../..') # go two back
BASE_DIR = '/kaggle/input/birdclef-2021/'
DIR_SPEC_IN = '/kaggle/input/birdclef-2021/train_short_audio/'
DIR_SPEC_OUT = '/kaggle/working/spec_temp/'
CSV_IN_TRAIN = '/kaggle/input/birdclef-2021/train_metadata.csv'
os.getcwd()

In [ ]:
# Main Data Class
class get_subset:

    def __init__(self):

        ''' SHORT TRAINING FILES '''
        self.__SHORTAUDIO__ = DIR_SPEC_IN
        # main short audio info CSV file
        self.pd_short_audio = pd.read_csv(BASE_DIR+'/train_metadata.csv')
        self.pd_short_audio['path'] = self.__SHORTAUDIO__ + "/" + self.pd_short_audio['primary_label'] + '/' + self.pd_short_audio['filename']
        
        ''' TRAINING SOUNDSCAPE FILES '''
        self.__SO_PATH_TR__ = BASE_DIR+'./train_soundscapes/'  # path to train soundcape files
        self.__SO_PATH_TE__ = BASE_DIR+'./test_soundscapes/'  # path to test soundcape files
        # main soundscape info CSV file
        path_soundscape_audio = BASE_DIR+'train_soundscape_labels.csv'   # read soundscape related CSV
        self.pd_scape = pd.read_csv(path_soundscape_audio)

        # list of filest to soundscape .ogg
        lst_sounds = os.listdir(self.__SO_PATH_TR__)
        self.PATH_SCAPE = [self.__SO_PATH_TR__ + i for i in lst_sounds]
        self.PATH_SCAPE.sort()

    # get all birds available in short sound files
    def get_short_labels(self):
        primary_labels = self.pd_short_audio.primary_label.unique()
        primary_labels.sort()
        return primary_labels

    # get various subsets of dataframe
    def get_bird_subset(self,name='acafly'):
        return self.pd_short_audio[self.pd_short_audio['primary_label'] == name].copy().reset_index()
    # get rating subset
    def get_rating_subset(self,rating=2):
        return self.pd_short_audio[self.pd_short_audio['rating'] == rating].copy().reset_index()
    # get bird & rating subset
    def get_bird_rating(self,name='acafly',rating=4):
        return self.pd_short_audio[(self.pd_short_audio['primary_label'] == name)&(self.pd_short_audio['rating'] == rating)].copy().reset_index()
    # # show name of primary label
    def primary_to_common(self,primary='cangoo'):
        specie = self.pd_short_audio[self.pd_short_audio['primary'] == primary].sample(1)
        return specie

subset = get_subset()
# Training soundscape csv (w/ weak labels)
# print(subset.pd_scape)
print('NUMBER OF NO CALL SEGMENTS AVAILABLE: ')
print(subset.pd_scape['birds'].value_counts()[:5])

In [ ]:
''' CREATE SPECTOGRAMS FROM SOUNDSCAPE DATA '''
# list_scape -> list of pathways to soundscape files (.ogg)
# remove_primary -> list of 'primary_labels' weak labels to be removed from data
def scape_to_spectogram(lst_scape,remove_primary=None):

    print('1. CREATING ALL TRAINING SPECTOGRAMS w/ WEAK LABELS')
    path_temp = DIR_SPEC_OUT + 'scape'
    # before prediction clear read folder if it exists
    if(os.path.exists(path_temp)):
        shutil.rmtree(path_temp)

    # for soundcape data, let's change cfg.cutoff to output all segment data
    cfg.cutoff = 600; lst_remove = []; lst_primary_removed = []
    with tqdm_notebook(total=len(lst_scape)) as pbar:
        # loop over all soundcape files available to us
        for path in lst_scape:
            pbar.update(1)

            # get dataframe we'll need 
            tid = path.split(os.sep)[-1].rsplit('_', 1)[0].rsplit('_', 1)[0]   # get soundscape id
            pname = path.split(os.sep)[-1].split('.ogg')[0]               # get soundscape id
            df_temp = subset.pd_scape[subset.pd_scape.audio_id == int(tid)] # get local soundcape df

            # 1. [MAKE] Get all soundscape spectograms for one soundscape .ogg file 
            get_spectrograms(path,tid,path_temp)

            # 2. [REMOVE] identify & remove spectograms (add pathways) with specified primary weak labels
            for label in remove_primary:
                found = df_temp.loc[df_temp['birds'] == label]
                if(not found.empty):
                    time_id = found.seconds.unique()
                    for time in time_id:
    #                     lst_remove.append('\\'+tid+'\\'+pname+'_'+str(time)+'.png')
                        lst_remove.append('/'+tid+'/'+pname+'_'+str(time)+'.png')
                        lst_primary_removed.append(label)    
    
    # [REMOVE] remove all files associated with specie
    print("2. REMOVING SEGMENTS W/ SPECIFIED 'PRIMARY_LABEL' WEAK LABEL")
    with tqdm_notebook(total=len(lst_remove)) as pbar:
        ii=-1
        for file in lst_remove:
            pbar.update(1)
#             print(f'{file} : {lst_primary_removed[ii]}')
            os.remove(path_temp+file)
            
''' CREATE SPECTOGRAMS '''
# Create Spectograms & Remove Primary Weak Labels

remove_id = subset.get_short_labels().tolist() # get list of all primary_labels
lst_scape = subset.PATH_SCAPE # get list of soundscape files

scape_to_spectogram(lst_scape,remove_id)

In [ ]:
def get_path_list(TPATH):
    scape_files = [] 
    SCAPE_FOLDER = os.listdir(TPATH)
    for folder in SCAPE_FOLDER:
        onlyfiles = next(os.walk(TPATH+folder))[2] #dir is your directory path as string
        print(f'{TPATH+folder} : {len(onlyfiles)} files')
        files = os.listdir(TPATH+folder)
        for file in files:
            scape_files.append(file)
        
print('LIST OF AVAILABLE NOCALL SEGMENTS:')
get_path_list('/kaggle/working/spec_temp/scape/')

In [ ]:
# Get list of noise samples
tpath = '/kaggle/working/spec_temp/scape/11254/'
lst_show = [tpath+i for i in os.listdir(tpath)]

# plot grid of examples
show_grid(lst_show,5,6,figsize=(25,15))